In [46]:
import pandas as pd

# PDF Utilities
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
from pdfminer.pdfpage import PDFPage

# Utilities
from io import StringIO

In [2]:
# Defining the reader function
def convert_pdf_to_text(path):
    """Read PDF file and return a string with
    text"""
    resource_manager = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(resource_manager, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(resource_manager, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [3]:
def clean_text(text):
    """Recieve a text and clean all 
    characters that we don't need"""
    for word in words_to_replace:
        text = text.replace(word, " ")
    for number in range(1, 30):
        text = text.replace(" " * number, " ")
    return text

In [4]:
# Cleaning text for a better text munging
words_to_replace = [
    "\x0c", ".", ",", "\n",
    " i)", "ii", "iii", "iv)", "v)", "vi)", ")",
    " i ", " v ", " vi ", " iv ",
    " a ", " ante ", " en ", " sin ", " so ",
    " con ", " tras ", " mediante ", " versus ",
    " de ", " para ", " vía ", " desde ", " por ", " ha ", " la "
]

In [5]:
# Routes to pdf's files
paths = list()
for index in range(1, 13):
    paths.append(f"pdf{index}.pdf")

paths

['pdf1.pdf',
 'pdf2.pdf',
 'pdf3.pdf',
 'pdf4.pdf',
 'pdf5.pdf',
 'pdf6.pdf',
 'pdf7.pdf',
 'pdf8.pdf',
 'pdf9.pdf',
 'pdf10.pdf',
 'pdf11.pdf',
 'pdf12.pdf']

In [6]:
# Cleaning text
cleaned_texts = list()
for path in paths:
    text = convert_pdf_to_text(path)
    text = clean_text(text)
    text = text.replace("  ", " ")
    cleaned_texts.append(text)
cleaned_texts[0]

' Anuncio Política Monetaria Comunicado Prensa 2 agosto 2018 La Junta Gobierno del Banco México decidido mantener el objetivo Tasa Interés Interbancaria un día un nivel 7 75% Los indicadores disponibles al segundo trimestre 2018 sugieren que economía mundial continuó expandiéndose un ritmo relativamente elevado No obstante se incrementado divergencia el desempeño las principales economías avanzadas En particular contraste zona del euro Japón y el Reino Unido que han crecido debajo lo previsto Estados Unidos actividad económica aceleró su ritmo crecimiento parte respuesta al estímulo fiscal adoptado En un contexto baja holgura esto podría conducir mayores presiones sobre inflación Como se anticipaba Reserva Federal mantuvo cambio el rango objetivo tasa fondos federales su reunión agosto y reiteró su previsión aumentos graduales dicha tasa Cabe señalar que economía mundial está sujeta incertidumbre tanto el corto plazo como mayor medida el mediano plazo Ello debido los riesgos asociados 

In [41]:
# Phrases to seek
phrases_to_seek = [
    "menor volatilidad",
    "mayor volatilidad",
    "actividad económica",
    "economía",
    "perspectivas crediticias",
    "holgura",
    "inflación",
    "inflación subyacente",
    "finanzas públicas",
    "incertidumbre",
    "postura monetaria prudente"
]

if "perspectivas crediticias" in cleaned_texts[0]:
    print("Yeii")
else:
    print("nay")

nay


In [79]:
texts_lists = list()
for text in cleaned_texts:
    text_list = text.split(" ")
    texts_lists.append(text_list)

In [43]:
# Searching phrases in each text
count = 0
universal_counts = list()
for text_list in texts_lists:
    word_count = list()
    for phrase in phrases_to_seek:
        for index in range(len(text_list)):
            word_required = str()
            for number in range(len(phrase.split(" "))):
                try:
                    word_required += text_list[index + number] + " "
                except IndexError:
                    pass
                if (phrase + " ") == word_required:
                    count += 1
        word_count.append(count)
        count = 0
    universal_counts.append(word_count)

In [44]:
universal_counts

[[0, 0, 2, 10, 0, 4, 15, 2, 1, 6, 1],
 [0, 0, 2, 10, 0, 3, 20, 5, 1, 4, 1],
 [0, 0, 2, 11, 0, 2, 19, 3, 2, 4, 1],
 [0, 0, 3, 9, 0, 2, 21, 3, 2, 5, 1],
 [0, 0, 2, 9, 0, 3, 20, 3, 2, 4, 1],
 [0, 0, 2, 9, 0, 3, 22, 3, 2, 5, 1],
 [0, 1, 1, 9, 1, 5, 25, 7, 2, 5, 1],
 [0, 0, 3, 10, 1, 4, 19, 3, 1, 6, 1],
 [0, 0, 3, 8, 1, 5, 19, 4, 1, 6, 1],
 [0, 0, 2, 9, 1, 5, 17, 3, 1, 6, 1],
 [1, 0, 2, 8, 2, 5, 17, 2, 1, 7, 1],
 [0, 0, 2, 9, 2, 6, 15, 1, 2, 6, 1]]

In [77]:
word_count_df = pd.DataFrame({
    "menor volatilidad": [],
    "mayor volatilidad": [],
    "actividad económica": [],
    "economía": [],
    "perspectivas crediticias": [],
    "holgura": [],
    "inflación": [],
    "inflación subyacente": [],
    "finanzas públicas": [],
    "incertidumbre": [],
    "postura monetaria prudente": [],
})

for count in universal_counts:
    temp_df = pd.DataFrame({
        "menor volatilidad": [count[0]],
        "mayor volatilidad": [count[1]],
        "actividad económica": [count[2]],
        "economía": [count[3]],
        "perspectivas crediticias": [count[4]],
        "holgura": [count[5]],
        "inflación": [count[6]],
        "inflación subyacente": [count[7]],
        "finanzas públicas": [count[8]],
        "incertidumbre": [count[9]],
        "postura monetaria prudente": [count[10]],
    })
    word_count_df = word_count_df.append(temp_df, ignore_index=True).astype(int)

In [78]:
word_count_df

,menor volatilidad,mayor volatilidad,actividad económica,economía,perspectivas crediticias,holgura,inflación,inflación subyacente,finanzas públicas,incertidumbre,postura monetaria prudente
0,0,0,2,10,0,4,15,2,1,6,1
1,0,0,2,10,0,3,20,5,1,4,1
2,0,0,2,11,0,2,19,3,2,4,1
3,0,0,3,9,0,2,21,3,2,5,1
4,0,0,2,9,0,3,20,3,2,4,1
5,0,0,2,9,0,3,22,3,2,5,1
6,0,1,1,9,1,5,25,7,2,5,1
7,0,0,3,10,1,4,19,3,1,6,1
8,0,0,3,8,1,5,19,4,1,6,1
9,0,0,2,9,1,5,17,3,1,6,1
